In [1]:
import __init__

from src.utilities import train_test_validation_splits, \
                            prepare_train_test_val_seqs_by_batches, \
                            tensor2dataloader, load_full_meltome_FLIP_db, \
                            seek_UniprotID_association_Meltome_prots,\
                            download_UniprotID_Alphafold_Structures,\
                            get_guided_encoder_output,\
                            ESM2_IF_repr
                            
import pandas as pd
import numpy as np
    
import ipdb
path_meltome = '../datasets/FLIP/splits/meltome/splits/mixed_split.csv'
complete_meltome_db = '../datasets/FLIP/splits/meltome/full_dataset_sequences.fasta'

full_meltome_db = load_full_meltome_FLIP_db(complete_meltome_db)

splits_meltome = pd.read_csv(path_meltome, sep=',')

#import ipdb; ipdb.set_trace()

splits_meltome = seek_UniprotID_association_Meltome_prots(splits_meltome, full_meltome_db)
split_meltome = splits_meltome[splits_meltome['sequence'].str.len()>= 50]
train, val, test = train_test_validation_splits(splits_meltome)

print("Done")

Done


In [ ]:
def outlier_seeker(df_t, root_path, id, std=5):

    import ipdb; ipdb.set_trace()

    for org  in train.organism.unique():
        df_t = train[train.organism.str.contains(org)].sort_values(by='target',ascending=False)
        org_tm_threshold = df_t[df_t.organism.str.contains(org)].tm_organism.unique().item() + std
        df_t.target = df_t.target.astype('float')
        outliers = df_t.query('target > @org_tm_threshold')[['full_name', 'protein_id','target','tm_organism']].iloc[0:50]#[['full_name', 'target','tm_organism']]
        outliers.to_csv('outliers/'+org+'_'+id+'.csv', index=False, sep="\t")

#outlier_seeker(train, 'outliers/','train')

In [ ]:

from tqdm.auto import tqdm
import torch
from src.ESM2embeddings import ESM2embeddings
from src.ESM2_IF1_embeddings import ESM2_IF1_embeddings
from src.utilities import train_test_validation_splits, prepare_train_test_val_seqs_by_batches, tensor2dataloader,read_and_plot_loss_curves, cleanup_by_organism_and_correspondence_idx
from src.NeuralArchitectures import regressionHead
from src.Trainer_seq_struct import Trainer_Seq_Struct
from src.Trainer_seq_struct2 import Trainer_Seq_Struct2

from src.AdaptiveDataset import AdaptiveDataset

structuredir = '../datasets/Structures/'
meltome_struct_dir_labels = '../datasets/'
organism = 'coli'
keyword = 'lysate'

import os 
if not os.listdir(structuredir) and not os.path.isfile(meltome_struct_dir_labels+'Meltome_Splits_FLIPS_train.txt')\
                                and not os.path.isfile(meltome_struct_dir_labels+'Meltome_Splits_FLIPS_test.txt')\
                                and not os.path.isfile(meltome_struct_dir_labels+'Meltome_Splits_FLIPS_val.txt'): 
    
    print("Finding structures to proteins via Alphafold/PDB dbs..........") 
    
    download_UniprotID_Alphafold_Structures(train, structuredir, meltome_struct_dir_labels, 'Meltome_Splits_FLIPS_train')
    download_UniprotID_Alphafold_Structures(test, structuredir, meltome_struct_dir_labels, 'Meltome_Splits_FLIPS_test')
    download_UniprotID_Alphafold_Structures(val, structuredir, meltome_struct_dir_labels, 'Meltome_Splits_FLIPS_val')



Structinfo_Meltome_Splits_train = pd.read_csv( meltome_struct_dir_labels+'Meltome_Splits_FLIPS_train.txt', sep='\t').set_index(train.index)
Structinfo_Meltome_Splits_test = pd.read_csv( meltome_struct_dir_labels+'Meltome_Splits_FLIPS_test.txt', sep='\t').set_index(test.index)
Structinfo_Meltome_Splits_val = pd.read_csv( meltome_struct_dir_labels+'Meltome_Splits_FLIPS_val.txt', sep='\t').set_index(val.index)


#import ipdb; ipdb.set_trace()

train = pd.concat([train, Structinfo_Meltome_Splits_train], axis=1).query('Source=="alphafold"')
test = pd.concat([test, Structinfo_Meltome_Splits_test], axis=1).query('Source=="alphafold"')
val = pd.concat([val, Structinfo_Meltome_Splits_val], axis=1).query('Source=="alphafold"')


idxdf_idxfile_train, _ = cleanup_by_organism_and_correspondence_idx(train, ['coli_lysate','Caenorhabditis_elegans_lysate'], keyword)
idxdf_idxfile_test, _ = cleanup_by_organism_and_correspondence_idx(test, organism, keyword)
idxdf_idxfile_val, _ = cleanup_by_organism_and_correspondence_idx(val, ['coli_lysate','Caenorhabditis_elegans_lysate'], keyword)
    

In [ ]:
import torch
import src.IF_PiFold_embeddings  as PiFoldconf 
#from src.IF_PiFold_embeddings import PiFold_setup


args = PiFoldconf.get_parser()
exp = PiFoldconf.PiFold_setup(args)
svpath = '../ref_models/PiFold/results/'
exp.method.load_state_dict(torch.load(svpath+'checkpoint.pth'))

import torch, os, gc
import torch_geometric
import torch_sparse
from torch_geometric.nn import MessagePassing
import warnings
warnings.filterwarnings('ignore')

pt_batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ESM2 = ESM2embeddings(type_embedding = "facebook/esm2_t33_650M_UR50D", device = device, type_tool='FacebookESM2')

dir_pre_generated_embeddings = '../prepro_embeddings/ESM2_single_embeddings_nobatches/'

path_train= dir_pre_generated_embeddings+'train_set_embeddings_ESM2_with_StructAssociation.pth'
path_test= dir_pre_generated_embeddings+'test_set_embeddings_ESM2_with_StructAssociation.pth'
path_val= dir_pre_generated_embeddings+'val_set_embeddings_ESM2_with_StructAssociation.pth'

path_IFtrain= dir_pre_generated_embeddings+'train_set_IFPiFoldembeddings_ESM2_StructAssociation.pth'
path_IFtest= dir_pre_generated_embeddings+'test_set_IFPiFoldembeddings_ESM2_StructAssociation.pth'
path_IFval= dir_pre_generated_embeddings+'val_set_IFPiFoldembeddings_ESM2_StructAssociation.pth'

In [ ]:
print ("Creating Embeddings...")   

train_embeddings = ESM2.esm2embedding(train, device, layer_index=33, pt_batch_size=1, folder_path=dir_pre_generated_embeddings+'Meltome/train/', correspondence_idx_df_file=idxdf_idxfile_train) 
test_embeddings = ESM2.esm2embedding(test, device, layer_index=33, pt_batch_size=1, folder_path=dir_pre_generated_embeddings+'Meltome/test/', correspondence_idx_df_file=idxdf_idxfile_test) 
val_embeddings = ESM2.esm2embedding(val, device, layer_index=33, pt_batch_size=1, folder_path=dir_pre_generated_embeddings+'Meltome/val/', correspondence_idx_df_file=idxdf_idxfile_val)



IF_embeddings_train = exp.IF_repr(train, folder_path='../prepro_embeddings/PiFold_if1_embeddings/Meltome_train', correspondence_idx_df_file=idxdf_idxfile_train)
IF_embeddings_test = exp.IF_repr(test, folder_path='../prepro_embeddings/PiFold_if1_embeddings/Meltome_test', correspondence_idx_df_file=idxdf_idxfile_test)
IF_embeddings_val = exp.IF_repr(val, folder_path='../prepro_embeddings/PiFold_if1_embeddings/Meltome_val', correspondence_idx_df_file=idxdf_idxfile_val)


print('starting UMAP mapping ....')
import umap, umap.plot
from pprint import pprint
#import ipdb; ipdb.set_trace()


ESM2train_UMAPproj = umap.UMAP(n_components=2).fit(train_embeddings.numpy())
ESM2trainlabels =  pd.Categorical(train.organism).codes

ESM2test_UMAPproj = umap.UMAP(n_components=2).fit(test_embeddings.numpy())
ESM2testlabels =  pd.Categorical(test.organism).codes

ESM2val_UMAPproj = umap.UMAP(n_components=2).fit(val_embeddings.numpy())
ESM2vallabels =  pd.Categorical(val.organism).codes


print("ESM2 EMBEDDINGS UMAP PLOTS:\n\n")
pprint(f' ESM2 categories on training data: \n { pd.Categorical(train.organism).categories.tolist()}')
umap.plot.points(ESM2train_UMAPproj, labels=ESM2trainlabels, color_key_cmap='Paired', background='black' )

pprint(f'\n\n ESM2 categories on test data: \n { pd.Categorical(test.organism).categories.tolist()}')
umap.plot.points(ESM2test_UMAPproj, labels=ESM2testlabels, color_key_cmap='Paired', background='black' )

pprint(f'\n\n ESM2 categories on validation data: \n { pd.Categorical(val.organism).categories.tolist()}')
umap.plot.points(ESM2val_UMAPproj, labels=ESM2vallabels, color_key_cmap='Paired', background='black' )


"""
print("PiFold EMBEDDINGS UMAP PLOTS:\n\n")
print(f' PiFold categories on training data: \n { pd.Categorical(train.organism)}')
umap.plot.points(ESM2train_UMAPproj, labels=ESM2trainlabels, color_key_cmap='Paired', background='black' )

print(f' PiFold categories on test data: \n { pd.Categorical(test.organism)}')
umap.plot.points(ESM2test_UMAPproj, labels=ESM2testlabels, color_key_cmap='Paired', background='black' )

print(f' PiFold categories on validation data: \n { pd.Categorical(val.organism)}')
umap.plot.points(ESM2val_UMAPproj, labels=ESM2vallabels, color_key_cmap='Paired', background='black' )
"""